# Feature extraction and embeddings

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np


## A. Preparation de donnees

In [41]:
df = pd.read_csv('./spooky.csv')

In [43]:
df

,id,text,author,tokens_space,tokens_rule,tokens_subword,entities,pos_tags,lemmatized,stemmed,great_count,impossible_count,sentiment
0,id26305,process however afforded means ascertaining di...,EAP,"['process', 'however', 'afforded', 'means', 'a...","['process', 'however', 'afforded', 'means', 'a...","['process', 'however', 'afforded', 'means', 'a...",[],"[('process', 'NOUN'), ('however', 'ADV'), ('af...",process however afforded mean ascertaining dim...,process howev afford mean ascertain dimens dun...,0,0,-0.5216
1,id17569,never occurred fumbling might mere mistake,HPL,"['never', 'occurred', 'fumbling', 'might', 'me...","['never', 'occurred', 'fumbling', 'might', 'me...","['never', 'occurred', 'fu', '##mbling', 'might...",[],"[('never', 'ADV'), ('occurred', 'VERB'), ('fum...",never occurred fumbling might mere mistake,never occur fumbl might mere mistak,0,0,-0.3400
2,id11008,left hand gold snuff box capered hill cutting ...,EAP,"['left', 'hand', 'gold', 'snuff', 'box', 'cape...","['left', 'hand', 'gold', 'snuff', 'box', 'cape...","['left', 'hand', 'gold', 's', '##nu', '##ff', ...","[('gold snuff', 'ORG'), ('hill cutting manner ...","[('left', 'VERB'), ('hand', 'NOUN'), ('gold', ...",left hand gold snuff box capered hill cutting ...,left hand gold snuff box caper hill cut manner...,1,0,0.9246
3,id27763,lovely spring looked windsor terrace sixteen f...,MWS,"['lovely', 'spring', 'looked', 'windsor', 'ter...","['lovely', 'spring', 'looked', 'windsor', 'ter...","['lovely', 'spring', 'looked', 'windsor', 'ter...","[('sixteen', 'CARDINAL'), ('years', 'DATE')]","[('lovely', 'ADJ'), ('spring', 'NOUN'), ('look...",lovely spring looked windsor terrace sixteen f...,love spring look windsor terrac sixteen fertil...,0,0,0.9460
4,id12958,finding nothing else even gold superintendent ...,HPL,"['finding', 'nothing', 'else', 'even', 'gold',...","['finding', 'nothing', 'else', 'even', 'gold',...","['finding', 'nothing', 'else', 'even', 'gold',...",[],"[('finding', 'VERB'), ('nothing', 'PRON'), ('e...",finding nothing else even gold superintendent ...,find noth els even gold superintend abandon at...,0,0,-0.8078
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19574,id17718,could fancied looked eminent landscape painter...,EAP,"['could', 'fancied', 'looked', 'eminent', 'lan...","['could', 'fancied', 'looked', 'eminent', 'lan...","['could', 'fan', '##cie', '##d', 'looked', 'em...",[],"[('could', 'AUX'), ('fancied', 'VERB'), ('look...",could fancied looked eminent landscape painter...,could fanci look emin landscap painter built b...,0,0,0.0000
19575,id08973,lids clenched together spasm,EAP,"['lids', 'clenched', 'together', 'spasm']","['lids', 'clenched', 'together', 'spasm']","['lids', 'clenched', 'together', 'spa', '##sm']",[],"[('lids', 'NOUN'), ('clenched', 'VERB'), ('tog...",lid clenched together spasm,lid clench togeth spasm,0,0,0.0000
19576,id05267,mais il faut agir say frenchman never faints o...,EAP,"['mais', 'il', 'faut', 'agir', 'say', 'frenchm...","['mais', 'il', 'faut', 'agir', 'say', 'frenchm...","['mai', '##s', 'il', 'fa', '##ut', 'ag', '##ir...","[('mais il', 'PERSON')]","[('mais', 'PROPN'), ('il', 'PROPN'), ('faut', ...",mais il faut agir say frenchman never faint ou...,mai il faut agir say frenchman never faint out...,0,0,0.0000
19577,id17513,item news like strikes us coolly received,EAP,"['item', 'news', 'like', 'strikes', 'us', 'coo...","['item', 'news', 'like', 'strikes', 'us', 'coo...","['item', 'news', 'like', 'strikes', 'us', 'coo...",[],"[('item', 'NOUN'), ('news', 'NOUN'), ('like', ...",item news like strike u coolly received,item news like strike us coolli receiv,0,0,0.0000


In [49]:
df = df[['id', 'text', 'author', 'tokens_subword']]

In [51]:
df

,id,text,author,tokens_subword
0,id26305,process however afforded means ascertaining di...,EAP,"['process', 'however', 'afforded', 'means', 'a..."
1,id17569,never occurred fumbling might mere mistake,HPL,"['never', 'occurred', 'fu', '##mbling', 'might..."
2,id11008,left hand gold snuff box capered hill cutting ...,EAP,"['left', 'hand', 'gold', 's', '##nu', '##ff', ..."
3,id27763,lovely spring looked windsor terrace sixteen f...,MWS,"['lovely', 'spring', 'looked', 'windsor', 'ter..."
4,id12958,finding nothing else even gold superintendent ...,HPL,"['finding', 'nothing', 'else', 'even', 'gold',..."
...,...,...,...,...
19574,id17718,could fancied looked eminent landscape painter...,EAP,"['could', 'fan', '##cie', '##d', 'looked', 'em..."
19575,id08973,lids clenched together spasm,EAP,"['lids', 'clenched', 'together', 'spa', '##sm']"
19576,id05267,mais il faut agir say frenchman never faints o...,EAP,"['mai', '##s', 'il', 'fa', '##ut', 'ag', '##ir..."
19577,id17513,item news like strikes us coolly received,EAP,"['item', 'news', 'like', 'strikes', 'us', 'coo..."


## B. Encodage de la variable à predire (facultatif)

In [54]:
le = LabelEncoder()
df['author_encoded'] = le.fit_transform(df['author'])

/tmp/ipykernel_13655/2425679132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['author_encoded'] = le.fit_transform(df['author'])


In [56]:
df

,id,text,author,tokens_subword,author_encoded
0,id26305,process however afforded means ascertaining di...,EAP,"['process', 'however', 'afforded', 'means', 'a...",0
1,id17569,never occurred fumbling might mere mistake,HPL,"['never', 'occurred', 'fu', '##mbling', 'might...",1
2,id11008,left hand gold snuff box capered hill cutting ...,EAP,"['left', 'hand', 'gold', 's', '##nu', '##ff', ...",0
3,id27763,lovely spring looked windsor terrace sixteen f...,MWS,"['lovely', 'spring', 'looked', 'windsor', 'ter...",2
4,id12958,finding nothing else even gold superintendent ...,HPL,"['finding', 'nothing', 'else', 'even', 'gold',...",1
...,...,...,...,...,...
19574,id17718,could fancied looked eminent landscape painter...,EAP,"['could', 'fan', '##cie', '##d', 'looked', 'em...",0
19575,id08973,lids clenched together spasm,EAP,"['lids', 'clenched', 'together', 'spa', '##sm']",0
19576,id05267,mais il faut agir say frenchman never faints o...,EAP,"['mai', '##s', 'il', 'fa', '##ut', 'ag', '##ir...",0
19577,id17513,item news like strikes us coolly received,EAP,"['item', 'news', 'like', 'strikes', 'us', 'coo...",0
